In [94]:
import sys, os
import json
import requests
import time
import datetime as dt
from datetime import date, timedelta, datetime

from itertools import repeat
import itertools

import numpy as np
import pandas as pd
import statistics as st
import scipy.stats as ss
from random import randint

import matplotlib.pyplot as plt
import seaborn as sns

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics

from fastai.tabular import *
import warnings 


## Load the data sets

In [123]:
warnings.filterwarnings('ignore')
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
df_store = pd.read_csv("../data/store.csv")
df_sample_sub= pd.read_csv("../data/sample_submission.csv")


len(df_train),len(df_test)

(1017209, 41088)

## Exploratory Data Analysis

In [149]:
print(df_train.shape)
train.head()

(844392, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [125]:
df_train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [126]:
print(df_test.shape)
test.head() 

(41088, 8)


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [127]:
df_test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [128]:
print(df_store.shape)
df_store.head()

(1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [129]:
df_store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [130]:
df_store.PromoInterval.value_counts()

Jan,Apr,Jul,Oct     335
Feb,May,Aug,Nov     130
Mar,Jun,Sept,Dec    106
Name: PromoInterval, dtype: int64

## Data Preparation / Feature Engineering

Pre-proccessing steps to the Rossman train, test and store data sets 

In [175]:
print(df_train.StateHoliday.unique())
print(df_test.StateHoliday.unique())

['0' 'a' 'b' 'c' 0]
['0' 'a']


In [132]:
df_train.Date.unique()

array(['2015-07-31', '2015-07-30', '2015-07-29', '2015-07-28',
       '2015-07-27', '2015-07-26', '2015-07-25', '2015-07-24',
       '2015-07-23', '2015-07-22', '2015-07-21', '2015-07-20',
       '2015-07-19', '2015-07-18', '2015-07-17', '2015-07-16',
       '2015-07-15', '2015-07-14', '2015-07-13', '2015-07-12',
       '2015-07-11', '2015-07-10', '2015-07-09', '2015-07-08',
       '2015-07-07', '2015-07-06', '2015-07-05', '2015-07-04',
       '2015-07-03', '2015-07-02', '2015-07-01', '2015-06-30',
       '2015-06-29', '2015-06-28', '2015-06-27', '2015-06-26',
       '2015-06-25', '2015-06-24', '2015-06-23', '2015-06-22',
       '2015-06-21', '2015-06-20', '2015-06-19', '2015-06-18',
       '2015-06-17', '2015-06-16', '2015-06-15', '2015-06-14',
       '2015-06-13', '2015-06-12', '2015-06-11', '2015-06-10',
       '2015-06-09', '2015-06-08', '2015-06-07', '2015-06-06',
       '2015-06-05', '2015-06-04', '2015-06-03', '2015-06-02',
       '2015-06-01', '2015-05-31', '2015-05-30', '2015-

In [134]:
df_train['Date']= pd.to_datetime(train['Date'])

In [135]:
df_train.Date

0         2015-07-31
1         2015-07-31
2         2015-07-31
3         2015-07-31
4         2015-07-31
             ...    
1017204   2013-01-01
1017205   2013-01-01
1017206   2013-01-01
1017207   2013-01-01
1017208   2013-01-01
Name: Date, Length: 1017209, dtype: datetime64[ns]

In [137]:
print ()
print ("-Over those two years, {} is the number of times that different stores closed on given days."
       .format(df_train[(df_train.Open == 0)].count()[0]))
print ()
print ("-From those closed events, {} times occured because there was a school holiday. "
       .format(df_train[(df_train.Open == 0) & (df_train.SchoolHoliday == 1)&(df_train.StateHoliday == '0') ].count()[0]))
print ()
print ("-And {} times it occured because of either a bank holiday or easter or christmas."
       .format(df_train[(df_train.Open == 0) &
         ((df_train.StateHoliday == 'a') |
          (df_train.StateHoliday == 'b') | 
          (df_train.StateHoliday == 'c'))].count()[0]))
print ()
print ("-But interestingly enough, {} times those shops closed on days for no apparent reason when no holiday was announced. In fact, those closings were done with no pattern whatsoever and in this case from 2013 to 2015 at almost any month and any day."
       .format(df_train[(df_train.Open == 0) &
         (df_train.StateHoliday == "0")
         &(df_train.SchoolHoliday == 0)].count()[0]))
print ()


-Over those two years, 172817 is the number of times that different stores closed on given days.

-From those closed events, 2263 times occured because there was a school holiday. 

-And 30140 times it occured because of either a bank holiday or easter or christmas.

-But interestingly enough, 121482 times those shops closed on days for no apparent reason when no holiday was announced. In fact, those closings were done with no pattern whatsoever and in this case from 2013 to 2015 at almost any month and any day.



In [139]:
#we will analyse only open stores since a close store yield a profit of 0.
df_train=df_train.drop(df_train[(df_train.Open == 0) & (df_train.Sales == 0)].index)

In [140]:
df_train = df_train.reset_index(drop=True) #making sure the indexes are back to [0,1,2,3 etc.] 

In [141]:
print ("Our new training set has now {} rows ".format(df_train.shape[0]))

Our new training set has now 844392 rows 


### Check for seasonality in both training and test sets - are the seasons similar between these two groups? 
    spring runs from March 1 to May 31;
    summer runs from June 1 to August 31;
    fall (autumn) runs from September 1 to November 30; and.
    winter runs from December 1 to February 28 (February 29 in a leap year)


In [107]:
season= train['Date']

In [108]:
ses= [y.quarter for y in season]
a= pd.array(ses)
a.unique()

<IntegerArray>
[3, 2, 1, 4]
Length: 4, dtype: Int64

In [109]:
 # winter a==1, spring a==2,  summer a==3,  fall  a==4
train[a==1]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
136030,1,2,2015-03-31,6206,589,1,1,0,1
136031,2,2,2015-03-31,10107,850,1,1,0,1
136032,3,2,2015-03-31,11299,862,1,1,0,1
136033,4,2,2015-03-31,14451,1724,1,1,0,1
136034,5,2,2015-03-31,7692,659,1,1,0,0
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


### Check for distribution in both training and test sets - are the promotions distributed similarly between these two groups

In [182]:
df_train.Promo.describe()

count    844392.000000
mean          0.446352
std           0.497114
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: Promo, dtype: float64

In [147]:
df_test.Promo.describe()

count    41088.000000
mean         0.395833
std          0.489035
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: Promo, dtype: float64

### Check & compare sales behavior before, during and after holidays

In [153]:
df_train.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [200]:
all_holiday = rom.dfdict['train'][(rom.dfdict['train'].StateHoliday == 'a')| (rom.dfdict['train'].StateHoliday == 'b') | (rom.dfdict['train'].StateHoliday == 'c')]  
pd.unique(all_holiday.StateHoliday)

array(['a', 'b', 'c'], dtype=object)

In [201]:
non_holiday = rom.dfdict['train'][ (rom.dfdict['train'].StateHoliday == 0)| (rom.dfdict['train'].StateHoliday == '0')]  
pd.unique(non_holiday.StateHoliday)

array(['0', 0], dtype=object)

In [202]:
all_holiday_sales = all_holiday.groupby('Date').sum()['Sales']

In [ ]:
all_holiday_sales.count()

### Find out any seasonal (Christmas, Easter etc) purchase behaviours,

### What can you say about the correlation between sales and number of customers?

In [180]:
corr_S_P=df_train[['Sales','Customers']]
corr_S_P.corr()             

,Sales,Customers
Sales,1.000000,0.823597
Customers,0.823597,1.000000


### How does promo affect sales? Are the promos attracting more customers? How does it affect already existing customers?

### Could the promos be deployed in more effective ways? Which stores should promos be deployed in?

### Trends of customer behavior during store open and closing times

### Which stores are opened on all weekdays? How does that affect their sales on weekends? 

### Check how the assortment type affects sales

In [116]:
class FetchPrepareData():
    def __init__(self):
        self.dfdict = {}
        self.dfdict['train'] = self.get_train_data()
        self.dfdict['test'] = self.get_test_data()
        self.dfdict['sample'] = self.get_sample_data()
        self.dfdict['store'] = self.get_store_data()
        
    def get_train_data(self,name='train'):
        filename = f'../data/{name}.csv'        
        return pd.read_csv(filename)
    
    def get_test_data(self,name='test'):
        filename = f'../data/{name}.csv'        
        return pd.read_csv(filename)
        
    def get_store_data(self,name='store'):
        filename = f'../data/{name}.csv'        
        return pd.read_csv(filename)  
    
    def get_sample_data(self,name='sample_submission'):
        filename = f'../data/{name}.csv'        
        return pd.read_csv(filename)  
        
    def df_to_prophet(self,name='train', 
                timecol="Date",
                targetcol="Sales",
                ftransform=None, 
                prefilter={}, 
                postfilter={},
                rs = '1D'):
        '''
        prepare data frame to prophet modelling
            dfin - input data frame
            timecol - the column name for time/date
            prefilter - a dictionary that contains column_name:value
            postfilter - a dictionary that contains column_name:value
            ftransform - a function to apply after prefiltering takes dfin as input
            rs - unit of time to resample time column 
        '''
        if name in ['train','test','sample']:
            df = self.dfdict[name].copy()
        else:
            print("only name=['train','test','sample'] are allowed")
            return pd.DataFrame()
        
        df['ts'] =  pd.to_datetime(df[timecol]).dt.tz_localize(None)
        df['ts'] = df['ts'].dt.to_pydatetime()

        #apply pre-filter
        for k, v in prefilter.items():
            print(f'Applying pre transform filter with column={k}, value={v}')
            df = df[df[k]==v]

        #transform
        if ftransform is not None:
            print(f'Applying functional transformation ..')
            df = ftransform(df)

        #apply post filter
        for k, v in prefilter.items():
            print(f'Applying post transform filter with column={k}, value={v}')
            df = df[df[k]==v]

        df = df.reset_index().set_index('ts').resample(rs).sum() 
        df = df.reset_index()

        df = df[['ts', targetcol]]
        df = df.rename(columns={"ts": "ds", targetcol: "y"})
        df = df.dropna()
        df.ds = pd.Series([v.to_pydatetime() for v in df.ds], dtype=object)
        return df

def plot_prophet(tsdf, 
                 changepoint_prior_scale=0.001, 
                 seasonality_prior_scale=1.0, 
                 periods=10, 
                 split=0.8 ):
    
    model = (Prophet(changepoint_prior_scale=changepoint_prior_scale, 
                    seasonality_prior_scale=seasonality_prior_scale, 
                    interval_width=0.95, 
                    daily_seasonality=True, 
                    weekly_seasonality=True, 
                    yearly_seasonality=False) \
            .add_seasonality(name='monthly', period=30.5, fourier_order=5) \
            .fit(tsdf)
            )
    
    future = model.make_future_dataframe(periods)
    forecast = model.predict(future)
    components = model.plot_components(forecast)

    forecast.ds = pd.Series([v.to_pydatetime() for v in forecast.ds], dtype=object)

    split = 0.8
    threshold_date_train = tsdf.ds[ int(len(tsdf.ds)*split) ]
    threshold_date_forecast = forecast.ds[ int(len(forecast.ds)*split) ]

    forecast_train = forecast[ threshold_date_forecast >= forecast.ds ]
    forecast_test = forecast[ threshold_date_forecast < forecast.ds ]
    df_train = tsdf[ threshold_date_train >= tsdf.ds ]
    df_test = tsdf[ threshold_date_train < tsdf.ds ]

    fig, ax = plt.subplots(figsize=(20,10))
    sns.set_style('darkgrid', {'axes.facecolor': '.9'})
    sns.set_palette(palette='deep')
    sns_c = sns.color_palette(palette='deep')

    ax.fill_between( x=forecast['ds'], y1=forecast['yhat_lower'], y2=forecast['yhat_upper'],
        color=sns_c[2], alpha=0.25, label=r'0.95 credible_interval')

    sns.scatterplot(x='ds', y='y', label='real historic data', data=df_train, ax=ax, color='black')
    sns.scatterplot(x='ds', y='y', label='real test data', data=df_test, ax=ax, color = 'red')
    sns.lineplot(x='ds', y='yhat', label='historic fit', data=forecast_train, ax=ax, color = 'blue')
    sns.lineplot(x='ds', y='yhat', label='future prediction', data=forecast_test, ax=ax, color = 'orange')
    ax.axvline(threshold_date_train, color=sns_c[3], linestyle='--', label='80% train-test data split')
    ax.legend(loc='upper left')
    ax.set_xlabel('Date')
    ax.set_ylabel('Engagement rate')
    ax.tick_params(axis='x', rotation=45)
    ax.set(title='Engagement rate model fit & prediction for Campaign ID ')
    
    return fig, components
    
    
#     model = Prophet(mcmc_samples=1000).fit(tsdf)
    
#     model = Prophet(seasonality_mode='multiplicative').add_seasonality('quarterly', period=91.25, fourier_order=8, 
#                        mode='additive').add_regressor('regressor', mode='additive').fit(tsdf)
    
#     model = Prophet(seasonality_mode='multiplicative').add_seasonality(name='monthly', period=30.5, fourier_order=5
#             ).add_seasonality(name='weekly', period=7, fourier_order=3
#             ).add_seasonality(name='daily', period=1, fourier_order=1
#             ).fit(tsdf)

In [121]:
# rom = FetchPrepareData()
# tsdf = rom.df_to_prophet(name='train',prefilter={'Store':1})
# print(tsdf.shape)
# tsdf.head()

In [119]:
# plot_prophet(tsdf, changepoint_prior_scale = 10,  periods = 5) 
# #              seasonality_prior_scale = 0.05